In [1]:
import adata

In [2]:
import pandas as pd

In [3]:
import time

In [4]:
from flask import Flask, render_template, jsonify

In [5]:
from threading import Thread, Lock
from concurrent.futures import ThreadPoolExecutor

In [6]:
import os

In [8]:
app = Flask(__name__)
current_data = pd.DataFrame()
data_lock = Lock()

stocks = [
    '600961', '601020', '600367', '002155', '301368',
    '603198', '605208', '603993', '000596', '600711'
]

In [9]:
def fetch_single_stock(code):
    try:
        df = adata.stock.market.get_capital_flow_min(stock_code=code)
        if not df.empty:
            df['trade_time'] = df['trade_time'].dt.strftime('%Y-%m-%d %H:%M:%S')
        return df
    except Exception as e:
        print(f"Error fetching {code}: {e}")
        return pd.DataFrame()

In [10]:
def fetch_data():
    global current_data
    with ThreadPoolExecutor(max_workers=10) as executor:
        dfs = list(executor.map(fetch_single_stock, stocks))
    
    valid_dfs = [df for df in dfs if not df.empty]
    if valid_dfs:
        new_data = pd.concat(valid_dfs).sort_values(by='trade_time', ascending=False)
        with data_lock:
            current_data = new_data
        
        os.system('cls' if os.name == 'nt' else 'clear')
        print(f"更新时间: {time.strftime('%Y-%m-%d %H:%M:%S')}")
        print(new_data.to_string(index=False))
    else:
        print("暂时无法获取数据")

In [ ]:
def scheduler():
    while True:
        fetch_data()
        time.sleep(60)

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/data')
def get_data():
    with data_lock:
        if current_data.empty:
            return jsonify([])
        data = current_data.to_dict('records')
    return jsonify(data)

if __name__ == '__main__':
    t = Thread(target=scheduler)
    t.daemon = True
    t.start()
    app.run(host='0.0.0.0', port=5000, debug=False)

In [13]:
import adata
import pandas as pd
import time
import os
from datetime import datetime
from html import escape

# 配置股票列表
STOCK_LIST = [
    '600961', '601020', '600367', '002155', '301368',
    '603198', '605208', '603993', '000596', '600711'
]

HTML_FILENAME = 'capital_flow_monitor.html'

def get_latest_data(stock_code):
    """获取单个股票的最新分钟级资金流数据"""
    try:
        df = adata.stock.market.get_capital_flow_min(stock_code=stock_code)
        return df.tail(1) if not df.empty else None
    except Exception as e:
        print(f"获取{stock_code}数据错误: {str(e)}")
        return None

def generate_html(df):
    """生成带自动刷新和样式的HTML文件"""
    safe_timestamp = escape(datetime.now().strftime('%Y-%m-%d %H:%M:%S')).replace(' ', '&nbsp;')
    
    html_template = f'''
    <!DOCTYPE html>
    <html>
    <head>
        <title>实时资金流监控</title>
        <meta charset="UTF-8">
        <meta http-equiv="refresh" content="60">
        <style>
            body {{ font-family: Arial, sans-serif; margin: 2rem; }}
            table {{ border-collapse: collapse; width: 100%; box-shadow: 0 1px 3px rgba(0,0,0,0.2); }}
            th, td {{ padding: 12px 15px; text-align: left; border-bottom: 1px solid #ddd; }}
            th {{ background-color: #4CAF50; color: white; }}
            tr:hover {{ background-color: #f5f5f5; }}
            .timestamp {{ color: #666; margin-bottom: 1rem; }}
        </style>
    </head>
    <body>
        <h2>股票资金流实时监控</h2>
        <div class="timestamp">最后更新时间：{safe_timestamp}</div>
        {df.to_html(index=False) if not df.empty else "暂无数据"}
    </body>
    </html>
    '''
    with open(HTML_FILENAME, 'w', encoding='utf-8') as f:
        f.write(html_template)

def main():
    while True:
        start_time = time.time()
        
        all_data = []
        for stock in STOCK_LIST:
            if (df := get_latest_data(stock)) is not None:
                all_data.append(df)
        
        combined_df = pd.concat(all_data) if all_data else pd.DataFrame()
        
        os.system('cls' if os.name == 'nt' else 'clear')
        print(f"最新数据更新时间：{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        if not combined_df.empty:
            print(combined_df.to_string(index=False))
        else:
            print("暂无有效数据")
        
        if not combined_df.empty:
            generate_html(combined_df)
            print(f"\n已更新网页文件：{HTML_FILENAME}")
        
        sleep_time = 60 - (time.time() - start_time) % 60
        time.sleep(max(sleep_time, 1))

if __name__ == "__main__":
    main()

获取301368数据错误: HTTPSConnectionPool(host='push2delay.eastmoney.com', port=443): Max retries exceeded with url: /api/qt/stock/fflow/kline/get?lmt=0&klt=1&fields1=f1,f2,f3,f7&fields2=f51,f52,f53,f54,f55,f56,f57,f58,f59,f60,f61,f62,f63,f64,f65&secid=0.301368 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001D69D4F2950>, 'Connection to push2delay.eastmoney.com timed out. (connect timeout=None)'))
最新数据更新时间：2025-03-09 19:03:51
stock_code          trade_time  main_net_inflow  sm_net_inflow  mid_net_inflow  lg_net_inflow  max_net_inflow
    600961 2025-03-07 15:00:00      200068680.0    -78071238.0    -121997469.0    -52131446.0     252200126.0
    601020 2025-03-07 15:00:00      139170149.0    -60725628.0     -78444546.0    -14526087.0     153696236.0
    600367 2025-03-07 15:00:00      141573203.0    -67207825.0     -74365441.0     -6491480.0     148064683.0
    002155 2025-03-07 15:00:00      165232423.0    -72837652.0     -92394745.0     53562208.0     1116

KeyboardInterrupt: 

In [ ]:
import adata
import pandas as pd
import time
import os
from datetime import datetime
from html import escape

# 配置股票列表和预测数据
STOCK_LIST = [
    '600961', '601020', '600367', '002155', '301368',
    '603198', '605208', '603993', '000596', '600711'
]

HTML_FILENAME = 'capital_flow_monitor.html'

def get_prediction_info(stock_code):
    """获取股票的预测信息"""
    return next((item for item in PREDICTION_DATA if item["stock_code"] == stock_code), None)

def get_latest_data(stock_code):
    """获取单个股票的最新分钟级资金流数据"""
    try:
        df = adata.stock.market.get_capital_flow_min(stock_code=stock_code)
        if df.empty:
            return None
            
        # 添加预测信息
        prediction = get_prediction_info(stock_code)
        if prediction:
            last_row = df.tail(1).copy()
            last_row['prediction'] = prediction["prediction"]
            last_row['stop_loss'] = prediction["stop_loss"]
            last_row['take_profit'] = prediction["take_profit"]
            return last_row
        return df.tail(1)
    except Exception as e:
        print(f"获取{stock_code}数据错误: {str(e)}")
        return None

def generate_html(df):
    """生成带自动刷新和样式的HTML文件"""
    safe_timestamp = escape(datetime.now().strftime('%Y-%m-%d %H:%M:%S')).replace(' ', '&nbsp;')
    
    # 生成带样式的HTML表格
    styled_html = df.to_html(
        index=False,
        columns=['stock_code', 'trade_time', 'prediction', 
                'stop_loss', 'take_profit', 'main_net_inflow'],
        formatters={
            'prediction': '{:.0f}'.format,
            'stop_loss': '{:.2f}'.format,
            'take_profit': '{:.2f}'.format,
            'main_net_inflow': '{:,.0f}'.format
        },
        classes='stock-table',
        escape=False
    )
    
    html_template = f'''
    <!DOCTYPE html>
    <html>
    <head>
        <title>实时资金流监控 - 今日选股</title>
        <meta charset="UTF-8">
        <meta http-equiv="refresh" content="60">
        <style>
            body {{ font-family: Arial, sans-serif; margin: 2rem; }}
            .header {{ 
                background: #4CAF50;
                color: white;
                padding: 1rem;
                border-radius: 5px;
                margin-bottom: 1rem;
            }}
            table {{ 
                border-collapse: collapse; 
                width: 100%; 
                box-shadow: 0 1px 3px rgba(0,0,0,0.2);
                margin-top: 1rem;
            }}
            th {{ 
                background-color: #f8f9fa;
                color: #212529;
                padding: 12px 15px;
                text-align: center;
                position: sticky;
                top: 0;
            }}
            td {{ 
                padding: 12px 15px;
                text-align: center;
                border-bottom: 1px solid #dee2e6;
            }}
            .prediction {{ 
                font-weight: bold;
                color: {lambda x: '#4CAF50' if x >=9 else '#FFC107' if x >=7 else '#F44336'};
            }}
            .timestamp {{ 
                color: #6c757d; 
                margin-bottom: 1rem;
                font-size: 0.9rem;
            }}
            tr:hover {{ background-color: #f8f9fa; }}
        </style>
    </head>
    <body>
        <div class="header">
            <h2>今日选股实时监控</h2>
            <div>监控股票：{', '.join(STOCK_LIST)}</div>
        </div>
        <div class="timestamp">最后更新时间：{safe_timestamp}</div>
        {styled_html}
    </body>
    </html>
    '''
    with open(HTML_FILENAME, 'w', encoding='utf-8') as f:
        f.write(html_template)

def main():
    while True:
        start_time = time.time()
        
        all_data = []
        for stock in STOCK_LIST:
            if (df := get_latest_data(stock)) is not None and not df.empty:
                all_data.append(df)
        
        combined_df = pd.concat(all_data) if all_data else pd.DataFrame()
        
        os.system('cls' if os.name == 'nt' else 'clear')
        print(f"最新数据更新时间：{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        if not combined_df.empty:
            print(combined_df[['stock_code', 'trade_time', 'prediction', 
                             'stop_loss', 'take_profit']].to_string(index=False))
        else:
            print("暂无有效数据")
        
        if not combined_df.empty:
            generate_html(combined_df)
            print(f"\n已更新网页文件：{HTML_FILENAME}")
        
        sleep_time = 60 - (time.time() - start_time) % 60
        time

SyntaxError: f-string: invalid syntax (1417789899.py, line 189)

In [2]:



PREDICTION_DATA = [
{
    "stock_code": "600961",
    "prediction": 10,
    "reason": "近1日主力资金净流入达2.01亿元，其中特大单净流入2.52亿元，显示主力资金在短期内强势介入。5日和10日主力资金净流入分别为2.16亿和1.58亿元，特大单持续流入，表明主力中长期布局意愿。K线显示，股票在2025-03-07放量涨停，单日换手率10.05%，成交量激增，价格突破近期盘整区间，形成强势反转信号。结合资金流向的持续性和K线的突破形态，预计未来短期上涨动能强劲。",
    "stop_loss": 8.2,
    "take_profit": 10.5,
    "analysis": "止损价设定为8.20元（上一交易日突破前的关键阻力位，现转为支撑位），因跌破此位置可能预示突破失败。止盈价设定为10.50元，基于近期主力资金持续流入强度及K线突破后的量度涨幅（参考盘整区间高度与资金推动势能）。当前AI预测为10分，反映极高上涨概率，但需防范短期获利回吐风险。",
    "timestamp": "2025-03-10 05:45:21"
  },
  {
    "stock_code": "601020",
    "prediction": 9,
    "reason": "该股票近期呈现显著资金推动型上涨特征：1. 资金流向方面，主力资金连续1/5/10日持续净流入（累计6.46亿元），其中特大单净流入占比超过85%，表明机构资金正在强势布局；2. K线形态上，过去10日股价实现翻倍式上涨，连续出现跳空缺口且每日收盘价均接近最高价，配合成交量阶梯式放大（近5日日均成交量较前期增长300%以上），形成典型量价齐升结构；3. 技术指标强化看涨信号，最近4个交易日连续触发涨停板，换手率维持在7%-17%的高活性区间，显示市场做多动能充沛。当前未出现顶部背离或异常放量滞涨信号，预计短期内仍具上涨惯性。",
    "stop_loss": 20.5,
    "take_profit": 26.8,
    "analysis": "止损价设定为20.5元（前一交易日收盘价），因该位置是短期关键支撑位，若跌破则可能标志上涨趋势中断；止盈价设定为26.8元，对应约18%潜在涨幅空间，此目标位综合考量了：1. 主力资金持续高强度流入（近10日特大单占比89%），2. 量价结构未现衰竭迹象（涨停缩量+跳空缺口未回补），3. 历史波动率测算（近10日平均振幅12%向上映射）。",
    "timestamp": "2025-03-10 05:45:24"
  },
  {
    "stock_code": "600367",
    "prediction": 9,
    "reason": "近1日、5日、10日主力资金持续净流入（分别1.47亿、1.44亿、1.526亿），且特大单净流入显著（1.53亿、1.66亿、1.666亿），表明机构资金强烈看好。K线显示近两日股价连续大涨（5.24%、10.02%），成交量激增（3月7日换手率9.89%），形成突破形态。尽管中小单资金净流出，但主力主导的买盘力量强劲，短期技术面与资金面共振，预示上涨动能充足。",
    "stop_loss": 14.3,
    "take_profit": 17.79,
    "analysis": "止损价设定为14.30元（3月7日最低价），因该位置为突破性大阳线的关键支撑位，若跌破则短期趋势可能反转。止盈价17.79元基于近期主力资金持续流入强度及技术形态量度涨幅（突破前平台后理论目标位），结合AI高评分对上涨动能的量化预期。",
    "timestamp": "2025-03-10 05:46:15"
  },
  {
    "stock_code": "002155",
    "prediction": 9,
    "reason": "资金流向显示主力资金持续强势介入，近1/5/10日特大单净流入分别达1.18亿、3.87亿、4.68亿，形成显著正向驱动。K线呈现量价齐升态势，近5日累计涨幅达22.3%，成交量逐级放大（3月7日换手率达10.29%），且连续突破关键价格位形成上升趋势。尽管中单资金持续流出显示部分筹码松动，但主力资金控盘力度强劲，结合短期技术面突破形态，预计上涨动能仍将持续。",
    "stop_loss": 20.7,
    "take_profit": 24.62,
    "analysis": "止损价格设定于3月7日最低价20.70元，该位置为短期关键支撑位，若跌破可能标志趋势反转。止盈价格基于主力资金持续流入及量价齐升态势，按上一收盘价22.38元的10%涨幅设定为24.62元，对应突破前高22.67元后的技术性目标位，反映上涨动能的延续预期。",
    "timestamp": "2025-03-10 05:46:29"
  },
  {
    "stock_code": "301368",
    "prediction": 9,
    "reason": "近期资金流向显示主力资金持续净流入，尤其是近1日和5日的特大单分别流入1.58亿和3.66亿元，表明大资金积极介入。K线显示股价在3月7日强势突破，单日涨幅15.2%，成交量显著放大至2260万股，换手率高达35.25%，形成量价齐升的看涨信号。尽管近10日主力净流入（3.83亿）较5日（5.15亿）有所放缓，但短期资金活跃度和价格动能显著增强，技术面呈现突破形态，预计未来仍有上行空间。",
    "stop_loss": 90.0,
    "take_profit": 110.0,
    "analysis": "止损价设为90.0元，基于3月7日大阳线的中间支撑位（开盘价84.0元与收盘价97.0元的中点），跌破可能预示短期动能衰竭；止盈价设为110.0元，参考近期高点101.04元上方阻力位及主力资金持续流入驱动的潜在上涨空间。技术面突破叠加资金活跃度提升，支撑上行预期，同时设置合理回撤保护。",
    "timestamp": "2025-03-10 05:47:00"
  },
  {
    "stock_code": "603198",
    "prediction": 8,
    "reason": "1. 资金流向分析：近1日主力资金净流入28,774,300元（含特大单12,438,500元、大单16,335,800元），表明短期大资金强势介入；近10日主力净流入71,327,800元（大单88,595,300元），中长期趋势向好。尽管近5日主力净流出，但小单资金显著承接（95,205,900元），市场情绪未明显衰退。\n2. K线趋势：过去10日股价呈震荡上升态势，3月7日单日涨幅5.45%且突破近期高点，成交量放大（11,438,600股）配合资金流入，形成量价齐升的强势信号。\n3. 风险点：中单资金持续净流出（10日累计-139,571,800元）显示部分中线资金离场，但主力与小单资金的合力或能支撑短期上行空间。综合判断未来1-2周有望延续反弹。",
    "stop_loss": 54.5,
    "take_profit": 62.5,
    "analysis": "1. **止损设定依据**：基于近期K线关键支撑位（3月7日低点54.28元）及中单资金持续流出风险，止损价设定在54.50元（略高于支撑位以避免短期波动触发）。\n2. **止盈设定依据**：参考近10日最高价（58.48元）与主力资金中长期流入趋势，结合量价突破信号，目标价设定为62.50元（较当前价57.82元约8%涨幅，对应主力资金推动的合理上行空间）。\n3. **综合逻辑**：短期资金强势介入与量价突破形成上行趋势，但需防范中线资金离场带来的回调风险。",
    "timestamp": "2025-03-10 05:47:20"
  },
  {
    "stock_code": "605208",
    "prediction": 9,
    "reason": "近10日主力资金持续净流入（3.21亿元），其中特大单净流入显著（3.67亿元），显示大资金积极布局。K线显示股价连续放量涨停，10日内涨幅达86.8%，换手率维持在较高水平（6.46%-22.88%），量价配合良好。尽管中单和大单短期有流出迹象（近1日中单-3844万元、大单-3185万元），但主力资金的主导地位和强劲的上涨动能（连续4日涨停）显示市场做多情绪强烈，短期内仍有惯性上冲空间。需警惕超买后的技术回调，但资金面支撑下，整体趋势偏乐观。",
    "stop_loss": 14.28,
    "take_profit": 17.28,
    "analysis": "止损价基于前一日（3月6日）收盘价14.28元设定，若股价跌破该支撑位，可能预示短期趋势反转。止盈价设定为当前收盘价15.71元的10%涨幅（17.28元），对应下一个潜在涨停板压力位。考虑到主力资金持续净流入与股价超买风险并存，此策略旨在锁定利润并控制回调风险。",
    "timestamp": "2025-03-10 05:47:36"
  },
  {
    "stock_code": "603993",
    "prediction": 8,
    "reason": "近1日和5日主力资金净流入显著（1.88亿和6.79亿），特大单资金持续强势流入（1.92亿和5.78亿），显示机构积极布局。过去10天K线呈现上升趋势，近期成交量放大且涨幅稳定（如3月6日涨3.65%、3月7日涨1.41%），换手率提升至1.86%，反映市场活跃度增加。尽管10日数据中大单资金有流出，但短期主力资金和价格动能支撑看涨预期，预计未来走势延续强势。",
    "stop_loss": 6.97,
    "take_profit": 7.55,
    "analysis": "止损价设为6.97元（基于上个交易日收盘价7.19元的3%跌幅），以防范短期波动风险并保护本金。止盈价设为7.55元（对应5%涨幅），因主力资金持续流入且K线呈现上升趋势，近期高点7.3元突破后有望进一步上行。该设置平衡了资金动能与技术阻力，符合AI对短期强势的预判。",
    "timestamp": "2025-03-10 05:47:45"
  },
  {
    "stock_code": "000596",
    "prediction": 8,
    "reason": "1. 资金流向显示短期（1日）主力资金净流入2231万元（其中特大单/大单主导），小单资金流出，暗示主力吸筹迹象；10日周期大单资金净流入10.62亿元形成强支撑，对冲了5日周期的短期流出压力。2. K线显示近期量价齐升，近两日涨幅达4.18%和1.76%，收盘价突破10日最高点，且成交量显著放大（03-06日换手率0.89%，03-07日量能维持高位），形成多头排列。3. 中单和小单在1/10日周期的分歧表明散户存在抛压，但主力资金的持续性流入和K线突破形态显示上涨动能占优。",
    "stop_loss": 170.0,
    "take_profit": 190.0,
    "analysis": "止损价格设定为170元，基于以下考量：1. 近两日K线低点（03-07日171.4元）与主力资金短期支撑位重合，若价格跌破170元将破坏多头结构；2. 该位置对应近期上涨波段（166.21元→176.2元）的50%斐波那契回撤位，技术面抛压可能加剧。止盈价格设定为190元，原因包括：1. 10日周期大单资金强支撑（106.24亿元）提供持续上涨动能；2. 当前股价突破历史密集成交区（03-03日174.94元高点），量能放大显示上方阻力较小，190元对应AI预测评分8分的中期估值区间上沿。",
    "timestamp": "2025-03-10 05:48:11"
  },
  {
    "stock_code": "600711",
    "prediction": 8,
    "reason": "近1/5/10日主力资金持续净流入（4891万/1.1亿/2504万），特大单和大单资金均呈现多日正向流入，表明机构资金持续布局。K线显示股价呈现阶梯式上涨趋势，近3个交易日连续放量突破前高（3月7日成交量较前日增长67%），收盘价创10日新高（5.14元）。主力与散户资金流向分化（小单持续净流出），结合量价齐升的技术形态，短期有望延续上涨动能。",
    "stop_loss": 4.92,
    "take_profit": 5.4,
    "analysis": "止损价设定为4.92元（较前收盘价-4.3%），基于近3个交易日平均低点（5.02/5.0/4.86元）与关键支撑位（2月24日跳空缺口上沿4.89元）叠加波动率缓冲。止盈价设定为5.40元（较前收盘价+5.0%），对应前期筹码密集区（2025年1月高点5.35-5.45元）及主力资金近5日净流入量级（1.1亿元）推升的合理目标区间。",
    "timestamp": "2025-03-10 05:48:19"
  }
]

def convert_prediction_data(prediction_list):
    """将原始预测数据列表转换为股票代码为键的字典格式"""
    return {
        item["stock_code"]: {
            "prediction": item["prediction"],
            "stop_loss": item["stop_loss"],
            "take_profit": item["take_profit"]
        }
        for item in prediction_list
    }

# 使用示例
PREDICTION_DICT = convert_prediction_data(PREDICTION_DATA)
print(PREDICTION_DICT)


{'600961': {'prediction': 10, 'stop_loss': 8.2, 'take_profit': 10.5}, '601020': {'prediction': 9, 'stop_loss': 20.5, 'take_profit': 26.8}, '600367': {'prediction': 9, 'stop_loss': 14.3, 'take_profit': 17.79}, '002155': {'prediction': 9, 'stop_loss': 20.7, 'take_profit': 24.62}, '301368': {'prediction': 9, 'stop_loss': 90.0, 'take_profit': 110.0}, '603198': {'prediction': 8, 'stop_loss': 54.5, 'take_profit': 62.5}, '605208': {'prediction': 9, 'stop_loss': 14.28, 'take_profit': 17.28}, '603993': {'prediction': 8, 'stop_loss': 6.97, 'take_profit': 7.55}, '000596': {'prediction': 8, 'stop_loss': 170.0, 'take_profit': 190.0}, '600711': {'prediction': 8, 'stop_loss': 4.92, 'take_profit': 5.4}}


In [ ]:
import adata
import pandas as pd
import time
import os
from datetime import datetime
from html import escape

HTML_FILENAME = 'capital_flow_monitor.html'

STOCK_LIST = [
    '600961', '601020', '600367', '002155', '301368',
    '603198', '605208', '603993', '000596', '600711'
]


def get_prediction_info(stock_code):
    """安全获取预测信息并验证数据完整性"""
    prediction = PREDICTION_DICT.get(stock_code, {})
    # 确保必要字段存在
    return {
        'prediction': prediction.get('prediction', 0),  # 默认0分
        'stop_loss': prediction.get('stop_loss', 0.0),
        'take_profit': prediction.get('take_profit', 0.0)
    }

def get_latest_data(stock_code):
    """获取并增强股票数据"""
    try:
        df = adata.stock.market.get_capital_flow_min(stock_code=stock_code)
        if df.empty:
            print(f"股票{stock_code}无可用数据")
            return None
            
        # 合并并验证预测数据
        prediction = get_prediction_info(stock_code)
        last_row = df.tail(1).copy()
        
        # 强制类型转换
        last_row['prediction'] = int(prediction['prediction'])
        last_row['stop_loss'] = float(prediction['stop_loss'])
        last_row['take_profit'] = float(prediction['take_profit'])
        
        return last_row.reset_index(drop=True)
    except Exception as e:
        print(f"处理{stock_code}数据错误: {str(e)}")
        return None

def fetch_realtime_data():
    """批量获取实时行情数据"""
    try:
        df1 = adata.stock.market.list_market_current(code_list=STOCK_LIST)
        if df1.empty:
            print("无实时数据返回")
            return pd.DataFrame()
        
        # 添加预测信息
        predictions = []
        for code in df1['stock_code']:
            predictions.append(get_prediction_info(code))
        
        # 合并数据
        pred_df = pd.DataFrame(predictions)
        return pd.concat([df1, pred_df], axis=1)
    except Exception as e:
        print(f"获取实时数据异常: {str(e)}")
        return pd.DataFrame()

def style_predictions(val):
    """增强样式处理函数"""
    try:
        score = int(val)
        if score >= 9:
            return 'color: #4CAF50; font-weight: bold;'
        elif score >= 7:
            return 'color: #FFC107; font-weight: bold;'
        return 'color: #F44336; font-weight: bold;'
    except:
        return ''  # 无效值时返回空样式

def generate_html(df):
    """生成带样式的HTML文件（增加数据验证）"""
    # 过滤无效数据
    valid_df = df.dropna(subset=['prediction']).reset_index(drop=True)
    
    safe_timestamp = escape(datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
    
    # 创建样式化表格
    styled_df = (
        valid_df.style
        .hide(axis='index')
        .format({
            'prediction': '{:.0f}',
            'stop_loss': '{:.2f}',
            'take_profit': '{:.2f}',
            'main_net_inflow': '¥{:,.0f}'
        })
        .applymap(style_predictions, subset=['prediction'])
    )
    
    html_template = f'''
    <!DOCTYPE html>
    <html>
    <head>
        <title>实时资金流监控 - 今日选股</title>
        <meta charset="UTF-8">
        <meta http-equiv="refresh" content="60">
        <style>
            /* 保持原有样式不变 */
        </style>
    </head>
    <body>
        <div class="header">
            <h2>今日选股实时监控</h2>
            <div>监控股票：{', '.join(STOCK_LIST)}</div>
        </div>
        <div class="timestamp">最后更新时间：{safe_timestamp}</div>
        {styled_df.to_html() if not valid_df.empty else "<p>暂无有效数据</p>"}
    </body>
    </html>
    '''
    with open(HTML_FILENAME, 'w', encoding='utf-8') as f:
        f.write(html_template)

def main():
    while True:
        start_time = time.time()
        
        all_data = []
        for stock in STOCK_LIST:
            if (df := get_latest_data(stock)) is not None:
                # 添加数据验证
                if not df.empty and pd.notna(df['prediction'].iloc[0]):
                    all_data.append(df)
        
        combined_df = pd.concat(all_data, ignore_index=True) if all_data else pd.DataFrame()
        
        os.system('cls' if os.name == 'nt' else 'clear')
        print(f"最新数据更新时间：{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        if not combined_df.empty:
            print(combined_df[['stock_code', 'trade_time', 'prediction', 
                             'stop_loss', 'take_profit']].to_string(index=False))
        else:
            print("暂无有效数据")
        
        if not combined_df.empty:
            generate_html(combined_df)
            print(f"\n已更新网页文件：{HTML_FILENAME}")
        
        sleep_time = 60 - (time.time() - start_time) % 60
        time.sleep(max(sleep_time, 1))

if __name__ == "__main__":
    main()

最新数据更新时间：2025-03-10 13:59:16
stock_code          trade_time  prediction  stop_loss  take_profit
    600961 2025-03-10 15:00:00          10       8.20        10.50
    601020 2025-03-10 15:00:00           9      20.50        26.80
    600367 2025-03-10 15:00:00           9      14.30        17.79
    002155 2025-03-10 15:00:00           9      20.70        24.62
    301368 2025-03-10 15:00:00           9      90.00       110.00
    603198 2025-03-10 15:00:00           8      54.50        62.50
    605208 2025-03-10 15:00:00           9      14.28        17.28
    603993 2025-03-10 15:00:00           8       6.97         7.55
    000596 2025-03-10 15:00:00           8     170.00       190.00
    600711 2025-03-10 15:00:00           8       4.92         5.40

已更新网页文件：capital_flow_monitor.html


C:\Users\charl\AppData\Local\Temp\ipykernel_38144\991573950.py:77: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  .applymap(style_predictions, subset=['prediction'])


KeyboardInterrupt: 

In [4]:
import adata
import pandas as pd
import time
import os
from datetime import datetime
from html import escape

HTML_FILENAME = 'stock_realtime_monitor.html'

STOCK_LIST = [
    '600961', '601020', '600367', '002155', '301368',
    '603198', '605208', '603993', '000596', '600711'
]


def get_prediction_info(stock_code):
    """安全获取预测信息"""
    return PREDICTION_DICT.get(stock_code, {
        'prediction': 0,
        'stop_loss': 0.0,
        'take_profit': 0.0
    })

def fetch_combined_data():
    """获取并合并实时行情和资金流数据"""
    try:
        # 获取实时行情数据
        realtime_df = adata.stock.market.list_market_current(code_list=STOCK_LIST)
        if realtime_df.empty:
            print("无实时行情数据")
            return pd.DataFrame()

        # 获取资金流数据
        capital_flow_data = []
        for code in STOCK_LIST:
            try:
                flow_df = adata.stock.market.get_capital_flow_min(stock_code=code)
                if not flow_df.empty:
                    latest_flow = flow_df.iloc[-1].to_dict()
                    latest_flow['stock_code'] = code
                    capital_flow_data.append(latest_flow)
            except Exception as e:
                print(f"获取{code}资金流数据失败: {str(e)}")
        
        # 合并数据
        capital_df = pd.DataFrame(capital_flow_data)
        if not capital_df.empty:
            merged_df = pd.merge(realtime_df, capital_df, on='stock_code', how='left')
        else:
            merged_df = realtime_df

        # 添加预测信息
        predictions = []
        for code in merged_df['stock_code']:
            predictions.append(get_prediction_info(code))
        pred_df = pd.DataFrame(predictions)
        
        return pd.concat([merged_df, pred_df], axis=1)
    
    except Exception as e:
        print(f"数据获取异常: {str(e)}")
        return pd.DataFrame()

def style_predictions(val):
    """预测分数样式"""
    try:
        score = int(val)
        if score >= 9:
            return 'background: #4CAF50; color: white; font-weight: bold;'
        elif score >= 7:
            return 'background: #FFC107; color: black; font-weight: bold;'
        return 'background: #F44336; color: white; font-weight: bold;'
    except:
        return ''

def generate_html(df):
    """生成带样式的监控页面"""
    # 数据清洗
    valid_df = df.copy()
    
    # 强制转换关键字段为数值类型
    numeric_columns = ['prediction', 'stop_loss', 'take_profit', 'price', 
                     'change_pct', 'main_net_inflow', 'amount']
    for col in numeric_columns:
        valid_df[col] = pd.to_numeric(valid_df[col], errors='coerce')
    
    # 过滤无效数据
    valid_df = valid_df.dropna(subset=['prediction']).reset_index(drop=True)
    safe_timestamp = escape(datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

    # 创建样式化表格（增加空值处理）
    styled_df = (
        valid_df.style
        .hide(axis='index')
        .format({
            'prediction': '{:.0f}',
            'stop_loss': '{:.2f}',
            'take_profit': '{:.2f}',
            'price': '{:.2f}',
            'change_pct': '{:.2f}%',
            'main_net_inflow': '¥{:,.0f}',
            'amount': '¥{:,.0f}'
        }, na_rep="N/A")  # 处理空值显示
        .applymap(style_predictions, subset=['prediction'])
        .set_properties(subset=numeric_columns, 
                      **{'text-align': 'right'})
    )

    html_template = f'''
    <!DOCTYPE html>
    <html>
    <head>
        <title>股票综合监控系统</title>
        <meta charset="UTF-8">
        <meta http-equiv="refresh" content="30">
        <style>
            body {{ font-family: Arial, sans-serif; margin: 20px; }}
            .header {{ 
                background: #2196F3; 
                color: white; 
                padding: 20px; 
                border-radius: 5px;
                margin-bottom: 20px;
            }}
            .timestamp {{ 
                color: #666; 
                margin-bottom: 15px;
                font-size: 0.9em;
            }}
            table {{
                width: 100%;
                border-collapse: collapse;
                box-shadow: 0 1px 3px rgba(0,0,0,0.12);
            }}
            th {{
                background: #f5f5f5;
                padding: 12px;
                text-align: left;
                border-bottom: 2px solid #ddd;
            }}
            td {{
                padding: 12px;
                border-bottom: 1px solid #eee;
            }}
            tr:hover {{ background: #f9f9f9; }}
            .numeric {{ text-align: right; }}
        </style>
    </head>
    <body>
        <div class="header">
            <h2>股票综合监控看板</h2>
            <div>监控标的：{', '.join(STOCK_LIST)}</div>
        </div>
        <div class="timestamp">最后更新时间：{safe_timestamp}</div>
        {styled_df.to_html() if not valid_df.empty else "<p>暂时无法获取数据</p>"}
    </body>
    </html>
    '''
    with open(HTML_FILENAME, 'w', encoding='utf-8') as f:
        f.write(html_template)

def main():
    while True:
        start_time = time.time()
        
        # 获取数据
        combined_df = fetch_combined_data()
        valid_df = combined_df.dropna(subset=['prediction']) if not combined_df.empty else pd.DataFrame()
        
        # 清屏并打印控制台信息
        os.system('cls' if os.name == 'nt' else 'clear')
        print(f"最新更新：{datetime.now().strftime('%H:%M:%S')}")
        if not valid_df.empty:
            display_cols = ['stock_code', 'short_name', 'price', 'change_pct', 
                           'main_net_inflow', 'prediction']
            print(valid_df[display_cols].to_string(index=False))
        else:
            print("暂无有效数据")
        
        # 生成网页文件
        if not valid_df.empty:
            generate_html(valid_df)
            print(f"已更新监控页面 -> {HTML_FILENAME}")
        
        # 精确间隔控制
        sleep_time = 30 - (time.time() - start_time) % 30
        time.sleep(max(sleep_time, 1))

if __name__ == "__main__":
    main()

最新更新：19:39:04
stock_code short_name  price change_pct  main_net_inflow  prediction
    600961       株冶集团  9.210       2.11      -99103567.0          10
    601020       华钰矿业 23.760       5.37     -422436840.0           9
    600367       红星发展 14.700      -3.67     -162171536.0           9
    002155       湖南黄金  21.88      -2.23      -88867053.0           9
    301368       丰立智能  89.20      -8.04     -112861115.0           9
    603198       迎驾贡酒 59.220       2.42       45323692.0           8
    605208        永茂泰 16.530       5.22      -13194459.0           9
    603993       洛阳钼业  7.810       8.62      955088462.0           8
    000596       古井贡酒 179.35       1.79       15316435.0           8
    600711       ST盛屯  5.400       5.06      124064504.0           8
已更新监控页面 -> stock_realtime_monitor.html


C:\Users\charl\AppData\Local\Temp\ipykernel_65656\701453132.py:104: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  .applymap(style_predictions, subset=['prediction'])


最新更新：19:39:36
stock_code short_name  price change_pct  main_net_inflow  prediction
    600961       株冶集团  9.210       2.11      -99103567.0          10
    601020       华钰矿业 23.760       5.37     -422436840.0           9
    600367       红星发展 14.700      -3.67     -162171536.0           9
    002155       湖南黄金  21.88      -2.23      -88867053.0           9
    301368       丰立智能  89.20      -8.04     -112861115.0           9
    603198       迎驾贡酒 59.220       2.42       45323692.0           8
    605208        永茂泰 16.530       5.22      -13194459.0           9
    603993       洛阳钼业  7.810       8.62      955088462.0           8
    000596       古井贡酒 179.35       1.79       15316435.0           8
    600711       ST盛屯  5.400       5.06      124064504.0           8
已更新监控页面 -> stock_realtime_monitor.html


C:\Users\charl\AppData\Local\Temp\ipykernel_65656\701453132.py:104: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  .applymap(style_predictions, subset=['prediction'])


获取600367资金流数据失败: HTTPSConnectionPool(host='push2delay.eastmoney.com', port=443): Max retries exceeded with url: /api/qt/stock/fflow/kline/get?lmt=0&klt=1&fields1=f1,f2,f3,f7&fields2=f51,f52,f53,f54,f55,f56,f57,f58,f59,f60,f61,f62,f63,f64,f65&secid=1.600367 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x00000188EF20A310>, 'Connection to push2delay.eastmoney.com timed out. (connect timeout=None)'))
最新更新：19:40:24
stock_code short_name  price change_pct  main_net_inflow  prediction
    600961       株冶集团  9.210       2.11      -99103567.0          10
    601020       华钰矿业 23.760       5.37     -422436840.0           9
    600367       红星发展 14.700      -3.67              NaN           9
    002155       湖南黄金  21.88      -2.23      -88867053.0           9
    301368       丰立智能  89.20      -8.04     -112861115.0           9
    603198       迎驾贡酒 59.220       2.42       45323692.0           8
    605208        永茂泰 16.530       5.22      -13194459.0           9
  

...

C:\Users\charl\AppData\Local\Temp\ipykernel_65656\701453132.py:104: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  .applymap(style_predictions, subset=['prediction'])


获取603993资金流数据失败: HTTPSConnectionPool(host='push2delay.eastmoney.com', port=443): Max retries exceeded with url: /api/qt/stock/fflow/kline/get?lmt=0&klt=1&fields1=f1,f2,f3,f7&fields2=f51,f52,f53,f54,f55,f56,f57,f58,f59,f60,f61,f62,f63,f64,f65&secid=1.603993 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x00000188EED713D0>, 'Connection to push2delay.eastmoney.com timed out. (connect timeout=None)'))
最新更新：19:46:22
stock_code short_name  price change_pct  main_net_inflow  prediction
    600961       株冶集团  9.750       5.86       89664573.0          10
    601020       华钰矿业 24.440       2.86       94040044.0           9
    600367       红星发展 15.380       4.63       52339353.0           9
    002155       湖南黄金  23.52       7.50      307973502.0           9
    301368       丰立智能  88.00      -1.35     -133552529.0           9
    603198       迎驾贡酒 60.640       2.40       55839439.0           8
    605208        永茂泰 15.870      -3.99       12710427.0           9
  

C:\Users\charl\AppData\Local\Temp\ipykernel_65656\701453132.py:104: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  .applymap(style_predictions, subset=['prediction'])


最新更新：19:47:01


KeyError: "['main_net_inflow'] not in index"

In [1]:
class StockScorer:

    def calculate_delta(self, main, big, medium, small, turnover_change, price_change):
        """
        计算实时动态分Δ
        :param main:     主力资金净流入（万元）
        :param big:      大户资金净流入（万元）
        :param medium:   中单资金净流入（万元）
        :param small:    小单资金净流入（万元）
        :param turnover_change: 换手率较前日变化（百分比，如+50表示增加50%）
        :param price_change:    价格涨跌幅（百分比，如-2表示下跌2%）
        :return: 动态分Δ（范围±5）
        """
        # 计算各指标得分
        main_score = self._calc_main(main)
        big_score = self._calc_big(big)
        medium_score = self._calc_medium(medium)
        small_score = self._calc_small(small)
        turnover_score = self._calc_turnover(turnover_change)
        price_score = self._calc_price(price_change)

        # 加权求和并限制范围
        delta = sum([
            main_score,     # 主力权重已包含在公式中
            big_score,      # 大户权重已包含在公式中
            medium_score,   # 中单权重已包含在公式中
            small_score,    # 小单权重已包含在公式中
            turnover_score, # 换手率权重已包含在公式中
            price_score     # 价格权重已包含在公式中
        ])
        return max(min(delta, 5), -5)

    def _calc_main(self, amount):
        """主力资金得分（权重30%）"""
        score = (amount / 100) * 0.3  # 每100万±0.3分
        return max(min(score, 3), -3)  # 限制±3分

    def _calc_big(self, amount):
        """大户资金得分（权重20%）"""
        score = (amount / 50) * 0.2   # 每50万±0.2分
        return max(min(score, 2), -2)  # 限制±2分

    def _calc_medium(self, amount):
        """中单资金得分（权重-10%）"""
        score = -(amount / 100) * 0.1 # 每100万-0.1分
        return max(score, -1)         # 下限-1分

    def _calc_small(self, amount):
        """小单资金得分（权重-10%）"""
        score = -(amount / 50) * 0.1  # 每50万-0.1分
        return max(score, -1)         # 下限-1分

    def _calc_turnover(self, change):
        """换手率得分（权重15%）"""
        if change >= 50:    # 增超50% → +1分
            return 1
        elif change <= -20: # 减少20% → -1分
            return -1
        else:
            return 0

    def _calc_price(self, change):
        """价格涨幅得分（权重15%）"""
        score = change * 0.5  # 每1%涨跌±0.5分
        return max(min(score, 3), -3)  # 限制±3分

    def get_final_score(self, delta):
        """计算最终评分（0-10）"""
        final = delta
        return max(min(round(final, 1), 10), 0)  # 保留1位小数

    def get_recommendation(self, final_score):
        """生成操作建议"""
        if final_score <= -3:
            return "强烈卖出"
        elif -2 <= final_score <= -1:
            return "卖出"
        elif 0 <= final_score < 2:
            return "中性观望"
        elif 2 <= final_score <= 4:
            return "买入"
        else:  # 5
            return "强烈买入"

if __name__ == "__main__":
    # 示例测试（对应文档案例）
    scorer = StockScorer()
    
    # 实时数据输入
    delta = scorer.calculate_delta(
        main=600,       # 主力+600万
        big=-100,       # 大户-100万
        medium=200,     # 中单+200万
        small=150,      # 小单+150万
        turnover_change=30,  # 换手率增30%
        price_change=-2      # 价格跌2%
    )
    
    final_score = scorer.get_final_score(delta)
    recommendation = scorer.get_recommendation(final_score)
    
    print(f"[动态分Δ] {delta:.1f}")
    print(f"[最终评分] {final_score}")
    print(f"[操作建议] {recommendation}")

[动态分Δ] -0.1
[最终评分] 0
[操作建议] 中性观望


In [ ]:
import schedule
import time
import pytz
import adata
import pandas as pd
from datetime import datetime, time as dt_time
#from StockScorer import StockScorer  # 假设原评分器代码保存为StockScorer.py


def get_today_date():
    """
    Returns today's date in 'YYYY-MM-DD' format using China Standard Time (CST).
    
    Returns:
        str: Today's date string in 'YYYY-MM-DD' format
    """
    # Get current time in China Standard Time
    china_tz = pytz.timezone('Asia/Shanghai')
    current_time_china = datetime.now(china_tz)
    
    # Format the date as YYYY-MM-DD
    formatted_date = current_time_china.strftime('%Y-%m-%d')
    
    return formatted_date


class StockMonitor:
    def __init__(self, stock_code='000001'):
        self.stock_code = stock_code
        self.scorer = StockScorer()
        
        # 缓存股本数据（避免频繁查询）
        self._cached_shares = None
        self._last_share_update = None
    def _get_capital_data(self):
        """获取资金流数据并转换为万元"""
        try:
            df = adata.stock.market.get_capital_flow_min(stock_code=self.stock_code)
            latest = df.iloc[-1]
            return {
                'main': float(latest['main_net_inflow']) / 10000,  # 强制转换为浮点
                'big': float(latest['lg_net_inflow']) / 10000,
                'medium': float(latest['mid_net_inflow']) / 10000,
                'small': float(latest['sm_net_inflow']) / 10000,
                'timestamp': latest['trade_time']   
            }
        except Exception as e:
            print(f"资金流数据获取失败: {str(e)}")
            return None


    
    def _get_market_data(self):
        """获取市场数据（新方案）"""
        try:
            # 获取实时价格数据
            price_df = adata.stock.market.list_market_current(code_list=[self.stock_code])
            if price_df.empty:
                return None
                
            latest_price = price_df.iloc[0]
            volume = float(latest_price['volume'])
            
            # 计算换手率
            turnover_ratio = self._calculate_turnover(volume)
            if turnover_ratio is None:
                return None
                
            return {
                'price_change': float(latest_price['change_pct']),  # 强制转换
                'turnover_change': float(turnover_ratio),
                'timestamp': datetime.now()
            }
        except Exception as e:
            print(f"市场数据获取异常: {str(e)}")
            return None
        
    
    def _get_share_structure(self):
        """获取最新流通A股股本"""
        try:
            # 每天只更新一次股本数据
            if self._cached_shares and (datetime.now() - self._last_share_update).days < 1:
                return self._cached_shares
            
            df = adata.stock.info.get_stock_shares(
                stock_code=self.stock_code,
                is_history=False  # 获取全部历史数据
            )
            
            if not df.empty:
                # 取最新日期的流通A股数据
                latest = df.iloc[0]
                self._cached_shares = latest['list_a_shares']
                self._last_share_update = datetime.now()
                return self._cached_shares
            return None
        except Exception as e:
            print(f"股本数据获取失败: {str(e)}")
            return None
        
    def _calculate_turnover(self, volume):
        """计算换手率"""
        shares = self._get_share_structure()
        if not shares or shares <= 0:
            print("错误：流通股本数据无效")
            return 0.0
            
        try:
            return round((volume / shares) * 100, 2)  # 换手率百分比
        except ZeroDivisionError:
            print("错误：流通股本为零")
            return 0.0
        
    def _run_scoring(self):
        """执行评分任务"""
        capital = self._get_capital_data()
        market = self._get_market_data()
        
        if capital and market:
            try:
                # 数据对齐检查
                #if (capital['timestamp'] - market['timestamp']).total_seconds() > 300:
                    #print("警告：资金流与市场数据时间不匹配")
                
                # 在_run_scoring中添加类型检查
                print(f"DEBUG - main类型: {type(capital['main'])}, 值: {capital['main']}")
                print(f"DEBUG - price_change类型: {type(market['price_change'])}, 值: {market['price_change']}")
                print(f"DEBUG - turnover_change类型: {type(market['turnover_change'])}, 值: {market['turnover_change']}")
                delta = self.scorer.calculate_delta(
                    main=capital['main'],
                    big=capital['big'],
                    medium=capital['medium'],
                    small=capital['small'],
                    turnover_change=market['turnover_change'],
                    price_change=market['price_change']
                )
                

                final_score = self.scorer.get_final_score(delta)
                recommendation = self.scorer.get_recommendation(final_score)
                
                # 存储结果（示例存储到CSV）
                result = {
                    'timestamp': datetime.now(),
                    "涨幅": market['price_change'],
                    "换手率": market['turnover_change'],
                    'main': capital['main'],
                    'big': capital['big'],
                    'medium': capital['medium'],
                    'small': capital['small'],
                    'delta': delta,
                    'final_score': final_score,
                    'recommendation': recommendation
                }
                pd.DataFrame([result]).to_csv('scores.csv', mode='a', header=False)
                
                print(f"[{datetime.now()}] 评分结果: {final_score} ({recommendation})")
                
            except Exception as e:
                print(f"评分计算失败: {str(e)}")

    def start_monitoring(self):
        """启动定时监控"""
        schedule.every(1).minutes.do(self._run_scoring)
        
        # 仅在交易时段运行（示例为A股时间）
        def during_trading_hours():
            now = datetime.now().time()
            is_weekday = datetime.now().weekday() < 5  # 0-4为周一到周五
            
            return is_weekday and (
                (dt_time(9, 30) <= now <= dt_time(11, 30) or 
                (dt_time(13, 0) <= now <= dt_time(15, 0))))
        
        while True:
            if during_trading_hours():
                schedule.run_pending()
            time.sleep(60)  # 每分钟检查一次

if __name__ == "__main__":
    monitor = StockMonitor(stock_code='000001')
    monitor.start_monitoring()

DEBUG - main类型: <class 'float'>, 值: -8225.691
DEBUG - price_change类型: <class 'float'>, 值: -0.08
DEBUG - turnover_change类型: <class 'float'>, 值: 0.68
[2025-03-13 14:28:04.899587] 评分结果: 0 (中性观望)
DEBUG - main类型: <class 'float'>, 值: -8225.691
DEBUG - price_change类型: <class 'float'>, 值: -0.08
DEBUG - turnover_change类型: <class 'float'>, 值: 0.68
[2025-03-13 14:29:08.052482] 评分结果: 0 (中性观望)
DEBUG - main类型: <class 'float'>, 值: -8225.691
DEBUG - price_change类型: <class 'float'>, 值: -0.08
DEBUG - turnover_change类型: <class 'float'>, 值: 0.68
[2025-03-13 14:30:11.079500] 评分结果: 0 (中性观望)
DEBUG - main类型: <class 'float'>, 值: -8225.691
DEBUG - price_change类型: <class 'float'>, 值: -0.08
DEBUG - turnover_change类型: <class 'float'>, 值: 0.68
[2025-03-13 14:31:14.288657] 评分结果: 0 (中性观望)


KeyboardInterrupt: 